<h1> Homework 2 - Data from the Web </h1>

In [3]:
## Importation of everything useful
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
#import html5lib
sns.set_context('notebook')

import requests
from bs4 import BeautifulSoup

In [4]:
parameters_page = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247')

In [5]:
parameters_html = parameters_page.text

In [6]:
soup = BeautifulSoup(parameters_html, "html.parser")

In [9]:
row_list = []
for parameter_cat_html in soup.find_all('select'):
    for option in parameter_cat_html.contents:
        row_dict = {'category':parameter_cat_html['name'], 'name':option.string, 'value':option['value']}
        row_list.append(row_dict)
parameters = pd.DataFrame(row_list)
parameters.head()

,category,name,value
0,ww_x_UNITE_ACAD,None,null
1,ww_x_UNITE_ACAD,Architecture,942293
2,ww_x_UNITE_ACAD,Chimie et génie chimique,246696
3,ww_x_UNITE_ACAD,Cours de mathématiques spéciales,943282
4,ww_x_UNITE_ACAD,EME (EPFL Middle East),637841336


<h2>Creating the base url</h2>

We found with Postman the base ur
l leading to the students list: http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.bhtml

To this page, we will add different parameters. Some of the parameters will not change for all the request, therefore we decided to hardcode them. These parameters are:
<ul>
<li>
<b>ww_x_GPS</b> = <em>-1</em><br>
This parameter varies during request but is not useful. It is probably linked to the gps position. It is was set to -1 which will accept everything.
</li>
<li>
<b>ww_i_reportModel</b> = <em>133685247</em>:<br>
Parameter to choose the type of files to take. We chose html files.
</li>
<li><b>ww_i_reportModelXsl</b> = <em>133685270</em>:<br>
Parameter to choose the type of files to take. We chose html files.
</li>
<li><b>ww_x_HIVERETE</b> = <em>null</em>:<br>
This parameter allow to choose between Winter and Spring semester. However, it is redondant, as Bachelor 1 can only happen in Winter semester, Bachelor 2 in Spring, Bachelor 3 in Winter, and so forth. Therefore we chose to eliminate this parameter by settings its value to null.
</li>
<li><b>ww_x_UNITE_ACAD</b> = <em>249847</em>:<br>
This value represents the "Informatique" section. As it was asked to only consider the data of this section, we have to use it all the time. If it was not the case, we could use the other technique.
</li>


</ul>

In [10]:
people_base_url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.bhtml?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_HIVERETE=null'

***Interesting parameters***<br>
Therefore we are left with two interesting parameters we want to vary:
the year ("periode academique") and the period ("periode pedagogique").

In [11]:
year_param_string = 'ww_x_PERIODE_ACAD'
period_param_string = 'ww_x_PERIODE_PEDAGO'

year_param = parameters.loc[parameters['category'] == year_param_string]
period_param = parameters.loc[parameters['category'] == period_param_string]

year_param = year_param.drop(year_param.iloc[0].name,axis=0, level=None)
period_param = period_param.drop(period_param.iloc[0].name,axis=0, level=None)
period_param = period_param[period_param['name'] != 'Mise à niveau']

In [12]:
year_param.head()

,category,name,value
21,ww_x_PERIODE_ACAD,2016-2017,355925344
22,ww_x_PERIODE_ACAD,2015-2016,213638028
23,ww_x_PERIODE_ACAD,2014-2015,213637922
24,ww_x_PERIODE_ACAD,2013-2014,213637754
25,ww_x_PERIODE_ACAD,2012-2013,123456101


In [13]:
period_param

,category,name,value
32,ww_x_PERIODE_PEDAGO,Bachelor semestre 1,249108
33,ww_x_PERIODE_PEDAGO,Bachelor semestre 2,249114
34,ww_x_PERIODE_PEDAGO,Bachelor semestre 3,942155
35,ww_x_PERIODE_PEDAGO,Bachelor semestre 4,942163
36,ww_x_PERIODE_PEDAGO,Bachelor semestre 5,942120
37,ww_x_PERIODE_PEDAGO,Bachelor semestre 5b,2226768
38,ww_x_PERIODE_PEDAGO,Bachelor semestre 6,942175
39,ww_x_PERIODE_PEDAGO,Bachelor semestre 6b,2226785
40,ww_x_PERIODE_PEDAGO,Master semestre 1,2230106
41,ww_x_PERIODE_PEDAGO,Master semestre 2,942192


In [14]:
#Construct a beautiful dataframe from a url
def create_df(url):
    
    # pd.read returns a list of dataframe
    # in our case, it returns a list of one dataframe,
    # so we select the first item of this list
    return pretify_df(pd.read_html(url)[0])
    

In [15]:
def pretify_df(data_frame):
    # We change the name of the columns by the value of the second line of each columns
    for i in range(0, len(data_frame.columns)):
        data_frame=data_frame.rename(columns = {i:data_frame.loc[1][i]})
        
    # We add a column for the period
    string = data_frame['Civilité'][0]
    splitedString = string.split(',')
    if False : #splitedString[0] != 'Informatique':
        return 'Not informatic section'
    else :
        data_frame['Period Academic'] = splitedString[1]
        periodPedagogic = splitedString[2].split(' ')[1:4]
        data_frame['Period pedagogic'] = ' '.join(periodPedagogic)

        # We drop the unuseful rows
        # And we drop the columns that contains only NaN
        # TODO: find a way to get the information of the first row (the row [0]), this row describe the year (period academic) and the period pedagogic
        data_frame = data_frame.drop(data_frame.index[[0,1]]).dropna(axis=1, how='all')
        return data_frame

In [16]:
def merged_inner(df1, df2):
    merged_inner = pd.merge(left=df1,right=df2, on='No Sciper')
    return merged_inner

In [18]:
def correcting_year(listOne, listSix):
    #For all Bachelor 1, we select the firt year of the column Period Academic
    for df in listOne:
        df["Period Academic"]=int(df['Period Academic'][2].split('-')[0].replace(" ", ""))

    #For all Bachelor 6, we select the second year of the column Period Academic
    for df in listSix:
        df["Period Academic"] = int(df['Period Academic'][2].split('-')[1])   

In [19]:
# Iterate for every academic year
# Input : 2 lists, one for Bachelor 1 and one for Bchelor 6
def final_Bachelor(listOne, listSix):
    finalResult = pd.DataFrame()
    
    correcting_year(listOne, listSix)
    for one in listOne:
        for six in listSix:
            
            result = duration_bachelor(one, six)

            # ignore_index=True since the different dataframes that will be append will have the same index
            # verify_integrity=True since we should not have the same student added several time (it makes no sense, a student can finish its Bachelor only one time)
            finalResult = finalResult.append(result,  ignore_index=True, verify_integrity=True)
    #finalResult['Duration of Bachelor']
            
    return finalResult

In [20]:
#Partition the data between male and female students and calculate a mean for particular gender
def mean_Bachelor(df, civilite):
    return df[df['Civilité'] == civilite]['Duration of Bachelor'].mean() 

In [ ]:
dfRes = 

*** Make the dataframe and save it on files: ***

In [ ]:
def url_param_str(param, value):
    return '&' + param + '=' + value

for i, year in year_param.iterrows():
    for j, period in period_param.iterrows():
            
            url_i = people_base_url + url_param_str(year.category, year.value) + url_param_str(period.category, period.value)
            file_name_i = './data/' + str(year['name']+period['name'])
            
            #print(file_name_i)
            print(url_i)
            try:
                df_i = create_df(url_i)
            except (ValueError, KeyError):
                print('-------> no file here!')
            else:
                df_i.to_pickle(file_name_i)

#test_dataframe = pd.read_pickle("./data/2016-2017Bachelor semestre 1")
#test_dataframe.head()

In [32]:
import glob

bachelors_sem1 = glob.glob('.\\data\\[0-9]*-[0-9]*Bachelor semestre 1')
bachelors_sem6 = glob.glob('.\\data\\[0-9]*-[0-9]*Bachelor semestre 6')

sem1_dfs = [pd.read_pickle(data_file) for data_file in bachelors_sem1]
sem6_dfs = [pd.read_pickle(data_file) for data_file in bachelors_sem6]

In [70]:
sem1dfs = [{'No Sciper': 1, 'year': 2000},
           {'No Sciper': 1, 'year': 2001},
          ]
sem2dfs = [{'No Sciper': 1, 'year': 2223},
           {'No Sciper': 1, 'year': 2222}
          ]

In [77]:
se1 = pd.DataFrame(sem1dfs)
se1 = se1.sort_values(['year'], ascending=True)
se1 = se1.drop_duplicates(subset='No Sciper', keep ='first')
se2 = pd.DataFrame(sem2dfs)
se2 = se2.sort_values(['year'], ascending=True)
se2 = se2.drop_duplicates(subset= 'No Sciper', keep ='last')
se1

,No Sciper,year
0,1,2000


In [66]:
res = pd.merge(se1, se2, on='No Sciper', how='inner')
res['d'] = res['year_y'] - res['year_x']
res

,No Sciper,year_x,year_y,d
0,1,2000,2222,222
1,1,2000,2223,223
2,1,2001,2222,221
3,1,2001,2223,222


In [33]:
sem1_dfs[4].head(1)

,Civilité,Nom Prénom,Statut,No Sciper,Period Academic,Period pedagogic
2,Monsieur,Aiulfi Loris Sandro,Présent,202293,2011-2012,Bachelor semestre 1


In [34]:
#res = final_Bachelor(sem1_dfs, sem6_dfs)
correcting_year(sem1_dfs, sem6_dfs)

In [35]:
sem1_dfs[4].head(1)

,Civilité,Nom Prénom,Statut,No Sciper,Period Academic,Period pedagogic
2,Monsieur,Aiulfi Loris Sandro,Présent,202293,2011,Bachelor semestre 1


In [52]:
sem1_dfs[0].head()

size : 1719


We fisrt concatenate all the dataframes for each semester in a big dataframe
then, we sort them by ascending Period Academic
and finally, for Bachelor 1, we drop duplicate and keep the first entry,
and for Bachelor 6, we drop duplicate and keep the last entry

In [88]:

bachelors_sem1 = pd.concat(sem1_dfs, ignore_index=True)
bachelors_sem1 = bachelors_sem1.sort_values(['Period Academic'], ascending=True)
bachelors_sem1 = bachelors_sem1.drop_duplicates(subset='No Sciper', keep='first')


bachelors_sem6 = pd.concat(sem6_dfs, ignore_index=True)
bachelors_sem6 = bachelors_sem6.sort_values(['Period Academic'], ascending=True)
bachelors_sem6 = bachelors_sem6.drop_duplicates(subset='No Sciper', keep='last')

bachelors_sem6.shape

(516, 9)

In [ ]:
We 

In [80]:
res = pd.merge(bachelors_sem1, bachelors_sem6, on='No Sciper', how='inner')
res.head(2)

,Civilité_x,Nom Prénom_x,Statut_x,No Sciper,Period Academic_x,Period pedagogic_x,Civilité_y,Ecole Echange,Filière opt.,Nom Prénom_y,Period Academic_y,Period pedagogic_y,Statut_y,Type Echange
0,Monsieur,Arévalo Christian,Présent,169569,2007,Bachelor semestre 1,Monsieur,NaN,NaN,Arévalo Christian,2010,Bachelor semestre 6,Présent,NaN
1,Monsieur,Obrist Damien,Présent,179194,2007,Bachelor semestre 1,Monsieur,Carnegie Mellon University Pittsburgh,NaN,Obrist Damien,2010,Bachelor semestre 6,Congé,Bilatéral


In [81]:
res.shape

(397, 14)

In [82]:
final_res = pd.DataFrame({'No Sciper': res['No Sciper'], 'Civilité' :res['Civilité_x'], 'Duration of Bachelor': res['Period Academic_y'] - res['Period Academic_x']})
len(final_res.index)

397

In [83]:
#average duration for male
mean_Bachelor(final_res, 'Monsieur')

3.4809782608695654

In [84]:
#average duration for female
mean_Bachelor(final_res, 'Madame')

3.310344827586207